In [131]:
import pandas as pd
import re
import nltk
import numpy as np
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from datetime import datetime
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tomma\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
#Importing the dataframe
df = pd.read_csv("results\Bitcoin_from_2021-12-22_23-59-49_to_2021-10-04_18-01-43.csv")
df = df.set_index('Datetime')
df.index = pd.to_datetime(df.index)

In [4]:
### Data Cleaning definition
pat1 = r'@[A-Za-z0-9]+' # this is to remove any text with @....
pat2 = r'https?://[A-Za-z0-9./]+'  # this is to remove the urls
combined_pat = r'|'.join((pat1, pat2)) 
pat3 = r'[^a-zA-Z]' # to remove every other character except a-z & A-Z
combined_pat2 = r'|'.join((combined_pat,pat3)) # we combine pat1, pat2 and pat3 to pass it in the cleaning steps

In [7]:
### Starting the cleaning
ps = PorterStemmer()
cleaned_tweets = []
for i in range(0, len(df)) :
    if( (i+1)%10000 == 0 ):
        print("Tweets %d of %d has been processed" % ( i+1, len(df['Text']) ))
    tweets = re.sub(combined_pat2,' ',df['Text'][i])
    tweets = tweets.lower()
    tweets = tweets.split()
    tweets = [ps.stem(word) for word in tweets if not word in set(stopwords.words('english'))]
    tweets = ' '.join(tweets)
    cleaned_tweets.append(tweets)

Tweets 10000 of 1000001 has been processed
Tweets 20000 of 1000001 has been processed
Tweets 30000 of 1000001 has been processed
Tweets 40000 of 1000001 has been processed
Tweets 50000 of 1000001 has been processed
Tweets 60000 of 1000001 has been processed
Tweets 70000 of 1000001 has been processed
Tweets 80000 of 1000001 has been processed
Tweets 90000 of 1000001 has been processed
Tweets 100000 of 1000001 has been processed
Tweets 110000 of 1000001 has been processed
Tweets 120000 of 1000001 has been processed
Tweets 130000 of 1000001 has been processed
Tweets 140000 of 1000001 has been processed
Tweets 150000 of 1000001 has been processed
Tweets 160000 of 1000001 has been processed
Tweets 170000 of 1000001 has been processed
Tweets 180000 of 1000001 has been processed
Tweets 190000 of 1000001 has been processed
Tweets 200000 of 1000001 has been processed
Tweets 210000 of 1000001 has been processed
Tweets 220000 of 1000001 has been processed
Tweets 230000 of 1000001 has been process

In [9]:
# Instantiate the sentiment intensity analyzer
vader = SentimentIntensityAnalyzer()

In [10]:
#Compute the score
df['cleaned_tweets'] = np.array(cleaned_tweets)
scores = df['cleaned_tweets'].apply(vader.polarity_scores).tolist()

In [14]:
# Convert the 'scores' list of dicts into a DataFrame
scores_df = pd.DataFrame(scores, index=df.index)# Join the DataFrames of the news and the list of dicts
parsed_and_scored_news = df.join(scores_df)

In [118]:
#Create a dataset with only the compound senitment and the datetime
compact_df = pd.DataFrame({'compound': parsed_and_scored_news['compound']})

In [119]:
# Convert the timestamp column to datetime objects
extended_dates = pd.DataFrame({'timestamp': pd.to_datetime(compact_df.index, errors='coerce')})
extended_dates['timestamp'].fillna("2099-01-01 00:00:00", inplace=True)
dates = extended_dates['timestamp'].dt.date.apply(lambda x: x.strftime('%Y-%m-%d'))

In [113]:
#Create the final df by grouping for dates
compact_df = pd.DataFrame({'compound': compact_df})
compact_df['date'] = np.array(dates)
final_df = compact_df.groupby(['date'])['compound'].sum()